# v3 Risk Estimator Notebook

This notebook reproduces the v3 risk estimator workflow: labeling high-risk, training a classifier, calibration, and local explanations.

Inputs:
- `v3/data_clean/v3_features_v1.csv` or `v3/data_clean/v3_features_v2.csv`


## 1) Setup

### Narrative commentary
This notebook mirrors the v3 model logic in the dashboard, but keeps the workflow explicit for review.


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

CWD = Path.cwd().resolve()
if (CWD / "v3").exists():
    REPO_ROOT = CWD
elif CWD.name == "notebooks" and (CWD.parent / "data_clean").exists():
    REPO_ROOT = CWD.parents[1]
else:
    REPO_ROOT = CWD

V3_DIR = REPO_ROOT / "v3"
DATA_CLEAN = V3_DIR / "data_clean"

DATA_CLEAN


## 2) Load features

### Narrative commentary
Choose the source (v1 or v2). v1 is closer to real data; v2 is synthetic and larger.


In [ ]:
SOURCE = "v2"  # change to "v1" if desired
path = DATA_CLEAN / f"v3_features_{SOURCE}.csv"

features = pd.read_csv(path)
features.head()


## 3) High-risk label

### Narrative commentary
High-risk is defined by a percentile cutoff on suicide_rate (default p80).


In [ ]:
cutoff = 0.80
threshold = features["suicide_rate"].quantile(cutoff)
features["high_risk"] = (features["suicide_rate"] >= threshold).astype(int)

threshold, features["high_risk"].mean()


## 4) Train a logistic model

### Narrative commentary
We use one-hot categorical features (region, income group, sex) and standardized numeric features.


In [ ]:
cat_cols = ["region_name", "income_group", "sex_name"]
num_cols = ["depression_dalys_rate", "addiction_death_rate", "selfharm_death_rate"]

X = features[cat_cols + num_cols]
y = features["high_risk"]

preprocessor = ColumnTransformer(
    [
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ]
)

model = LogisticRegression(max_iter=1000, class_weight="balanced")
base = Pipeline([("prep", preprocessor), ("model", model)])
base.fit(X, y)

calibrated = None
if y.nunique() > 1 and len(features) >= 60:
    calib_base = Pipeline([("prep", preprocessor), ("model", model)])
    try:
        calibrated = CalibratedClassifierCV(estimator=calib_base, method="isotonic", cv=3)
    except TypeError:
        calibrated = CalibratedClassifierCV(base_estimator=calib_base, method="isotonic", cv=3)
    calibrated.fit(X, y)

predictor = calibrated if calibrated is not None else base


## 5) Training diagnostics

### Narrative commentary
These metrics are training-only and serve as a quick sanity check.


In [ ]:
proba = predictor.predict_proba(X)[:, 1]
preds = (proba >= 0.5).astype(int)

metrics = {
    "accuracy": accuracy_score(y, preds),
    "auc": roc_auc_score(y, proba) if y.nunique() > 1 else np.nan,
    "brier": brier_score_loss(y, proba),
}
metrics


## 6) Calibration plot

### Narrative commentary
If calibrated, predicted probabilities should align more closely with observed rates.


In [ ]:
rel = pd.DataFrame({"proba": proba, "y": y})
rel["bin"] = pd.qcut(rel["proba"], q=8, duplicates="drop")
cal = rel.groupby("bin", observed=True).agg(mean_pred=("proba", "mean"), observed_rate=("y", "mean")).reset_index(drop=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=cal["mean_pred"], y=cal["observed_rate"], mode="markers+lines", name="Observed"))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", name="Ideal", line=dict(dash="dash", color="#6b6460")))
fig.update_layout(title="Reliability plot", xaxis_title="Mean predicted", yaxis_title="Observed rate")
fig


## 7) Counterfactual hints (10% reduction)

### Narrative commentary
This shows how the predicted probability changes if one feature is reduced by 10%.


In [ ]:
row = features.iloc[0].copy()
base_input = pd.DataFrame([row])[cat_cols + num_cols]
base_proba = float(predictor.predict_proba(base_input)[0][1])

cf_rows = []
for feature in num_cols:
    new_row = row.copy()
    new_row[feature] = max(0.0, float(new_row[feature]) * 0.9)
    new_proba = float(predictor.predict_proba(pd.DataFrame([new_row])[cat_cols + num_cols])[0][1])
    cf_rows.append({"feature": feature, "delta_probability": new_proba - base_proba})

pd.DataFrame(cf_rows)


## 8) Local feature contributions

### Narrative commentary
For a single row, we approximate contributions using standardized inputs and logistic coefficients.


In [ ]:
prep = base.named_steps["prep"]
clf = base.named_steps["model"]
feature_names = prep.get_feature_names_out()

x_trans = prep.transform(base_input)
if hasattr(x_trans, "toarray"):
    x_vec = x_trans.toarray()[0]
else:
    x_vec = np.asarray(x_trans)[0]

contrib = x_vec * clf.coef_.ravel()
contrib_df = pd.DataFrame({"feature": feature_names, "contribution": contrib})
contrib_df["abs"] = contrib_df["contribution"].abs()
contrib_df = contrib_df.sort_values("abs", ascending=False).head(10)

fig = px.bar(
    contrib_df.sort_values("contribution"),
    x="contribution",
    y="feature",
    orientation="h",
    title="Top local contributions",
)
fig


## 9) Notes

- v3 is a synthetic risk estimator, not a clinical model.
- Use these outputs to explain methodology, not individual outcomes.
